In [2]:
import numpy as np
import cv2
import time

def draw_hsv1(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)

    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return bgr

def draw_hsv2(flow):
    h, w = flow.shape[:2]
    hsv = np.zeros((h, w, 3), np.uint8)

    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[...,1] = 255
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return bgr

def dense_optical_flow(video_path, output_path):

    cap = cv2.VideoCapture(video_path)
    ret, old_frame = cap.read()
    # old_frame = cv2.resize(old_frame, (768,512))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 20, (old_frame.shape[1], old_frame.shape[0]), 10000)
    
    hsv = np.zeros_like(old_frame)
    hsv[..., 1] = 255
 
    old_frame = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    # old_frame = cv2.fastNlMeansDenoising(old_frame)

    while True:
        ret, new_frame = cap.read()
        # new_frame = cv2.resize(new_frame, (768,512))
        frame_copy = new_frame
        
        if not ret:
            break
        start = time.time()
        new_frame = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
        # new_frame = cv2.fastNlMeansDenoising(new_frame)

        flow = None
        flow = cv2.calcOpticalFlowFarneback(prev=old_frame,
                                      next=new_frame, flow=flow,
                                      pyr_scale=0.5, levels=3, winsize=5,
                                      iterations=3, poly_n=5, poly_sigma=1,
                                      flags=10)
            
        bgr = draw_hsv2(flow)

        end = time.time()
        if end != start:
            fps = 1 / (end-start)
        
        cv2.putText(frame_copy, f'FPS: {fps}',
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(bgr, f'FPS: {fps}',
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        cv2.imshow("frame", new_frame)
        cv2.imshow("optical flow", bgr)

        out.write(bgr)
        
        k = cv2.waitKey(5)
        if k == ord('q'):
            break
     
        old_frame = new_frame

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    

In [4]:
dense_optical_flow('test2.mp4', 'output.mp4')